In [1]:
import operator
import spacy
nlp = spacy.load("en_core_web_md")

In [2]:
docmap={}

In [ ]:
# irlist = ["data/wordintersection/score-val-top10-merged-val.tsv","data/wordintersection/score-val-val-trained.tsv"]
# irlist.extend(["data/wordunion/score-val-ir-top10-merged-val-wordunion.tsv","data/wordunion/score-val-val-trained-wordunion.tsv"])
# irlist.extend(["data/bagofwords/score-val-ir-top10-merged-bow-val.tsv","data/bagofwords/score-val-ir-not-merged-bow-val.tsv"])
# irlist.extend(["data/seq2seq/score-val-ir-top10-merged-val-seq2seq.tsv","data/seq2seq/score-val-ir-val-trained-seq2seq.tsv"])

# scorelist = ["data/wordintersection/score-val-top10-merged-val.tsv-score.tsv","data/wordintersection/score-val-val-trained.tsv-score.tsv"]
# scorelist.extend(["data/wordunion/score-val-ir-top10-merged-val-wordunion.tsv-score.tsv","data/wordunion/score-val-val-trained-wordunion.tsv-score.tsv"])
# scorelist.extend(["data/bagofwords/score-val-ir-top10-merged-bow-val.tsv-score.tsv","data/bagofwords/score-val-ir-not-merged-bow-val.tsv-score.tsv"])
# scorelist.extend(["data/seq2seq/score-val-ir-top10-merged-val-seq2seq.tsv-score.tsv","data/seq2seq/score-val-ir-val-trained-seq2seq.tsv-score.tsv"])

# picklist = ["data/wordintersection/ir-top10-val-merged-intersect.pickle","data/wordintersection/ir-val-trained-intersect.pickle"]
# picklist.extend(["data/wordunion/ir-top10-merged-val-union-merged.pickle","data/wordunion/val-trained-merged.pickle"])
# picklist.extend(["data/bagofwords/ir-top10-merged-bow.pickle","data/bagofwords/ir-not-merged-bow.pickle"])
# picklist.extend(["data/seq2seq/ir-top10-merged-val-seq2seq.pickle","data/seq2seq/ir-val-trained-seq2seq.pickle"])

In [15]:
irlist = ["data/wordintersection/score-test-top10-merged-test.tsv","data/wordintersection/score-test-test-trained.tsv"]
irlist.extend(["data/wordunion/score-test-ir-top10-merged-test-wordunion.tsv","data/wordunion/score-test-test-trained-wordunion.tsv"])
irlist.extend(["data/bagofwords/score-test-ir-top10-merged-bow-test.tsv","data/bagofwords/score-test-ir-not-merged-bow-test.tsv"])
irlist.extend(["data/seq2seq/score-test-ir-top10-merged-test-seq2seq.tsv","data/seq2seq/score-test-ir-test-trained-seq2seq.tsv"])

scorelist = ["data/wordintersection/score-test-top10-merged-test.tsv-score.tsv","data/wordintersection/score-test-test-trained.tsv-score.tsv"]
scorelist.extend(["data/wordunion/score-test-ir-top10-merged-test-wordunion.tsv-score.tsv","data/wordunion/score-test-test-trained-wordunion.tsv-score.tsv"])
scorelist.extend(["data/bagofwords/score-test-ir-top10-merged-bow-test.tsv-score.tsv","data/bagofwords/score-test-ir-not-merged-bow-test.tsv-score.tsv"])
scorelist.extend(["data/seq2seq/score-test-ir-top10-merged-test-seq2seq.tsv-score.tsv","data/seq2seq/score-test-ir-test-trained-seq2seq.tsv-score.tsv"])

picklist = ["data/wordintersection/ir-top10-test-merged-intersect.pickle","data/wordintersection/ir-test-trained-intersect.pickle"]
picklist.extend(["data/wordunion/ir-top10-merged-test-union-merged.pickle","data/wordunion/test-trained-merged.pickle"])
picklist.extend(["data/bagofwords/ir-top10-merged-bow.pickle","data/bagofwords/ir-not-merged-bow.pickle"])
picklist.extend(["data/seq2seq/ir-top10-merged-test-seq2seq.pickle","data/seq2seq/ir-test-trained-seq2seq.pickle"])

In [16]:
testlines = open("../data/orig/orig-ques-test.tsv").readlines()
labelsdict = {}
for line in testlines:
    line=line.strip().split("\t")
    qid = line[0]
    label = line[-1]
    labelsdict[qid]=int(label)
    choices = line[2:6]
    for index,choice in enumerate(choices):
        labelsdict[qid+":"+str(index)]=choice

In [17]:
from tqdm import tqdm

def get_doc(docs):
    if docs in docmap:
        return docmap[docs]
    docmap[docs] = nlp(docs)
    return docmap[docs]

def create_dataset(irtestlines,scorelines,labelsdict):
    ir_dataset_f2= {}
    for irline,scoreline in zip(irtestlines,scorelines):
        scoreline = scoreline.strip().split("\t")
        qidx = scoreline[0].split(":")
        score =float(scoreline[-1])
        qid = qidx[0]
        hid = qidx[1]
        qidx = qid+":"+hid
        if qidx not in ir_dataset_f2:
            ir_dataset_f2[qidx]=ir_dataset[qidx]
            ir_dataset_f2[qidx]["f2set"]={}
        irline = irline.strip().split("\t")
        f2 = irline[2]
        if len(f2.split(" ")) > 3:
            ir_dataset_f2[qidx]["f2set"][f2]=score

    for qidx,row in tqdm(ir_dataset_f2.items()):
        sorted_x = sorted(row["f2set"].items(), key=operator.itemgetter(1))
        row["f2set"] = [i for i in reversed(sorted_x)]
    return ir_dataset_f2

In [19]:
import pickle

for irfile,scorefile,pickfile in zip(irlist,scorelist,picklist):
    print("Current File:",scorefile+'.f2-reranked-repickled')
    irtestlines = open(irfile).readlines()
    scorelines = open(scorefile).readlines()
    fd = open(pickfile,"rb")
    ir_dataset = pickle.load(fd)
    ir_dataset_reranked = create_dataset(irtestlines,scorelines,labelsdict)
    with open(scorefile+'.no-reranked-repickled', 'wb+') as handle:
        pickle.dump(ir_dataset_reranked, handle, protocol=pickle.HIGHEST_PROTOCOL)

Current File: data/wordintersection/score-test-top10-merged-test.tsv-score.tsv.f2-reranked-repickled


100%|██████████| 2000/2000 [00:00<00:00, 38302.22it/s]


Current File: data/wordintersection/score-test-test-trained.tsv-score.tsv.f2-reranked-repickled


100%|██████████| 2000/2000 [00:00<00:00, 16137.28it/s]


Current File: data/wordunion/score-test-ir-top10-merged-test-wordunion.tsv-score.tsv.f2-reranked-repickled


...

100%|██████████| 2000/2000 [00:00<00:00, 49211.30it/s]


Current File: data/seq2seq/score-test-ir-top10-merged-test-seq2seq.tsv-score.tsv.f2-reranked-repickled


100%|██████████| 2000/2000 [00:00<00:00, 66521.87it/s]


Current File: data/seq2seq/score-test-ir-test-trained-seq2seq.tsv-score.tsv.f2-reranked-repickled


100%|██████████| 2000/2000 [00:00<00:00, 20791.41it/s]


In [24]:
ir_dataset_f2["8-201:2"]

{'hyp': 'A red-tailed hawk is searching for prey. It is most likely to swoop down on a gecko',
 'f1list': [['hawks eat lizards', '18.651140'],
  ['some birds are predators', '17.770297'],
  ['eagles eat fish', '17.040488'],
  ['eagles eat rabbits', '16.689134'],
  ['an example of camouflage is an organism looking like leaves', '14.846638'],
  ['an example of camouflage is when an organism looks like its environment',
   '14.472994'],
  ['birds sometimes eat insects', '14.119740'],
  ['some predators move quickly to catch prey', '13.912813'],
  ['a salamander eats insects', '6.660327'],
  ['camouflage can be used for hunting for prey', '6.566403']],
 'f2set': [('Baby red-tail hawk In their search for prey, redtails are often seen soaring our skies using updrafts and thermals.',
   0.9988596111643431),
  ('Red-tailed Hawk by Landon, age 10 If you see a bird that is eating a rattlesnake or mice, squirrel, rabbit or insect, it might be a red-tailed hawk.',
   0.9987876425662383),
  ('the r

In [12]:
import pickle
with open('data/wordunion/rerankedf2-ir-top10-merged-test-union-merged.pickled', 'wb+') as handle:
    pickle.dump(ir_dataset_f2, handle, protocol=pickle.HIGHEST_PROTOCOL)